In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import datetime
import calendar
from pandas.api.types import CategoricalDtype

In [4]:
#Import data
customers = pd.read_csv(r'https://raw.githubusercontent.com/PixarJunkie/dsc-672-capstone-project/main/data/olist_customers_dataset.csv')
locations = pd.read_csv(r'https://raw.githubusercontent.com/PixarJunkie/dsc-672-capstone-project/main/data/olist_geolocation_dataset.csv')
items = pd.read_csv(r'https://raw.githubusercontent.com/PixarJunkie/dsc-672-capstone-project/main/data/olist_order_items_dataset.csv')
payments = pd.read_csv(r'https://raw.githubusercontent.com/PixarJunkie/dsc-672-capstone-project/main/data/olist_order_payments_dataset.csv')
reviews = pd.read_csv(r'https://raw.githubusercontent.com/PixarJunkie/dsc-672-capstone-project/main/data/olist_order_reviews_dataset.csv')
orders = pd.read_csv(r'https://raw.githubusercontent.com/PixarJunkie/dsc-672-capstone-project/main/data/olist_orders_dataset.csv')
products = pd.read_csv(r'https://raw.githubusercontent.com/PixarJunkie/dsc-672-capstone-project/main/data/olist_orders_dataset.csv')

# Summary

In [7]:
datasets = [customers, items, payments, orders, products, locations, reviews]
col_names = ['customers', 'items', 'payments', 'orders', 'products', 'locations', 'reviews']

#To gather practical information about all datasets 
summary = pd.DataFrame({},)
summary['dataset']= col_names
summary['cols'] = [', '.join([col for col, null in df.isnull().sum().items() ]) for df in datasets]
summary['cols_no']= [df.shape[1] for df in datasets]
summary['null_no']= [df.isnull().sum().sum() for df in datasets]
summary['null_cols_no']= [len([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]
summary['null_cols'] = [', '.join([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]


summary.style.background_gradient(cmap='coolwarm')

,dataset,cols,cols_no,null_no,null_cols_no,null_cols
0,customers,"customer_id, customer_unique_id, customer_zip_code_prefix, customer_city, customer_state",5,0,0,
1,items,"order_id, order_item_id, product_id, seller_id, shipping_limit_date, price, freight_value",7,0,0,
2,payments,"order_id, payment_sequential, payment_type, payment_installments, payment_value",5,0,0,
3,orders,"order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date",8,4908,3,"order_approved_at, order_delivered_carrier_date, order_delivered_customer_date"
4,products,"order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date",8,4908,3,"order_approved_at, order_delivered_carrier_date, order_delivered_customer_date"
5,locations,"geolocation_zip_code_prefix, geolocation_lat, geolocation_lng, geolocation_city, geolocation_state",5,0,0,
6,reviews,"review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp",7,146532,2,"review_comment_title, review_comment_message"


In [8]:
#Merging Data
